In [1]:
# Importo las librerias que utilizaré
 
import pandas as pd
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bSoup
 
# ==============================================
 
# Funciones utilizadas 
 
def findMin(df, column):
    minimo = 100000000.0
    fecha = ''
    for idx in df.index:      # Por cada fila va a tomar el indice que sera almacenado en variable idx
        if float(df[column][idx]) < minimo:
            minimo = float(df[column][idx])
            fecha = df['fecha'][idx]
    return fecha, minimo
 
 
 
def findMax(df, column):
    maximo = -1000000000.0
    fecha = ''
    for idx in df.index:
        if float(df[column][idx]) > maximo:   # Si tomando el valor flotante de df['apertura_max' = nombre de columna][idx = numero de fila en la tabla] '1231231'  pasa a ser FLOTANTE
            maximo = float(df[column][idx])
            fecha = df['fecha'][idx]
    return fecha, maximo
 
# ===============================================
 
 
 
# Solicitud HTTP - GET = Consulta
URL = "https://es.finance.yahoo.com/quote/TEF/history?p=TEF&.tsrc=fin-srch"
rq = HTMLSession()    # Instancia de clase HTMLSession en variable rq
pagina = rq.get(URL)  # Uso la instancia de rq para realizar la solicitud y almacenarlo en variable pagina  (Almacena todo el código HTMl de la URL proporcionada)
 
# Codificamos el contenido
soup = bSoup(pagina.content, "html.parser")   # variable soup va a contener los resultados que reciba de BeatuifulSoup enviandole como parametros la pagina codificada y el tipo de decodificacion que se usara
 
# Creación de listas = Para las Columnas de las tabla (titulo de columna o encabezado)
fecha = []
apertura = []
apertura_max = []
apertura_min = []
cierre = []
volumen = []
 
# Query ()COMENTARIO  
elements = soup.find_all("tr", class_="BdT")  # ELEMENTS = LISTA []  Dentro de la variable elements voy a almacenar una busqueda dentro del html contenido en soup que sea su tag tr y su clase BdT
# CONTENIDO DE ELEMENTS
# [<tr ... . > CONTENIDO </tr>, <tr ... . > CONTENIDO </tr>, <tr ... . > CONTENIDO </tr>, <tr ... . > CONTENIDO </tr>, <tr ... . > CONTENIDO </tr>, <tr ... . > CONTENIDO </tr>,]
# print(elements)
 
 
# Llenado de listas desde html
for e in elements:              # por cada valor en la lista de elements, almacene el valor actual en la variable e
    row = e.find_all("span")      # variable row CONTENDRÁ de e, busqueda de todos los elementos HTML que su tag sea SPAN
  # Se debe verificar si la fecha no está repetida
  #   en caso de estar repetida, saltará a la siguiente línea
    if len(row) == 7:
        fecha.append(str(row[0].contents).replace("['","").replace("']",""))  
        apertura.append(str(row[1].contents).replace(',','.').replace("['","").replace("']","")) 
        apertura_max.append(str(row[2].contents).replace(',','.').replace("['","").replace("']",""))
        apertura_min.append(str(row[3].contents).replace(',','.').replace("['","").replace("']",""))
        cierre.append(str(row[4].contents).replace(',','.').replace("['","").replace("']",""))
        volumen.append(str(row[6].contents).replace('.','').replace("['","").replace("']",""))
 
 
# Creación de diccionario para pandas
dictionary = {
    "fecha": fecha,
    "apertura": apertura,
    "apertura_max": apertura_max,
    "apertura_min": apertura_min,
    "cierre": cierre,
    "volumen": volumen,
}
 
# ================ PANDAS
# Creación dataframe
data = pd.DataFrame(dictionary)
 
# Manipulación de datos
apMinFecha, apMinValue = findMin(data, 'apertura_min')
apMaxFecha, apMaxValue = findMax(data, 'apertura_max')
cierreMinFecha, cierreMinValue = findMin(data, 'cierre')
cierreMaxFecha, cierreMaxValue = findMax(data, 'cierre')
volumenMinFecha, volumenMinValue = findMin(data, 'volumen')
volumenMaxFecha, volumenMaxValue = findMax(data, 'volumen')
 
# Impresión de datos
print('\n* * * APERTURA * * *') 
print('El mínimo fue el '+str(apMinFecha)+' por valor de '+str(apMinValue)) 
print('El máximo fue el '+str(apMaxFecha)+' por valor de '+str(apMaxValue)+'\n') 
print('\n* * * CIERRE * * *') 
print('El mínimo fue el '+str(cierreMinFecha)+' por valor de '+str(cierreMinValue)) 
print('El máximo fue el '+str(cierreMaxFecha)+' por valor de '+str(cierreMaxValue)+'\n')
print('\n* * * VOLUMEN * * *') 
print('El mínimo fue el '+str(volumenMinFecha)+' por valor de '+str(volumenMinValue)) 
print('El máximo fue el '+str(volumenMaxFecha)+' por valor de '+str(volumenMaxValue)+'\n')






* * * APERTURA * * *
El mínimo fue el 26 sept 2022 por valor de 3.34
El máximo fue el 31 may 2022 por valor de 5.38


* * * CIERRE * * *
El mínimo fue el 26 sept 2022 por valor de 3.35
El máximo fue el 31 may 2022 por valor de 5.32


* * * VOLUMEN * * *
El mínimo fue el 13 jul 2022 por valor de 725700.0
El máximo fue el 03 jun 2022 por valor de 5644000.0

